<a href="https://colab.research.google.com/github/JanisJ2/jsc270-a4/blob/main/JSC270_Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Github link: https://github.com/JanisJ2/jsc270-a4 \
Group member:
- Christoffer Tan (1008740445)
- Janis Joplin (10097515051)


In [265]:
# Import necessary modules
from google.colab import files
import io
import sys
import re

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, roc_curve

from scipy.cluster.hierarchy import dendrogram, linkage

import seaborn as sns

import nltk
nltk.download('punkt')  # Tokenizer
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.porter import *

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [266]:
# Read the dataset from GitHub
!wget "https://raw.githubusercontent.com/JanisJ2/jsc270-a4/main/covid-tweets-train.csv"
!wget "https://raw.githubusercontent.com/JanisJ2/jsc270-a4/main/covid-tweets-test.csv"

--2024-03-29 19:45:55--  https://raw.githubusercontent.com/JanisJ2/jsc270-a4/main/covid-tweets-train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8991364 (8.6M) [text/plain]
Saving to: ‘covid-tweets-train.csv.7’

covid-tweets-train. 100%[===================>]   8.57M  --.-KB/s    in 0.07s   

2024-03-29 19:45:55 (121 MB/s) - ‘covid-tweets-train.csv.7’ saved [8991364/8991364]

--2024-03-29 19:45:55--  https://raw.githubusercontent.com/JanisJ2/jsc270-a4/main/covid-tweets-test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length

In [267]:
# Read the train data from CSV file
train_data = pd.read_csv('covid-tweets-train.csv')
train_data.columns = ['Label', 'Message', 'Sentiment']
train_data.head()

,Label,Message,Sentiment
0,0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,1
1,1,advice Talk to your neighbours family to excha...,2
2,2,Coronavirus Australia: Woolworths to give elde...,2
3,3,My food stock is not the only one which is emp...,2
4,4,"Me, ready to go at supermarket during the #COV...",0


In [269]:
def filter_missing_values(data):
    data.dropna(inplace=True)

In [270]:
# Clean missing values from train_data
filter_missing_values(train_data)

# Part I

## a)

In [271]:
# Given the a data set, it will return the proportion of the observations belong the a sentiment type
def proportion_of_sentiment(data, sentiment_type):
    return (data['Sentiment'] == sentiment_type).sum() / len(data['Sentiment'])

In [272]:
for i in range(3):
    print(f"The proportion of the train observations belonging to sentiment type {i}: {proportion_of_sentiment(train_data, str(i))}")

The proportion of the train observations belonging to sentiment type 0: 0.37414040288678835
The proportion of the train observations belonging to sentiment type 1: 0.18739824557140428
The proportion of the train observations belonging to sentiment type 2: 0.43841275241173183


## b)

In [273]:
def tokenize(data):
    data['Tokens'] = data['Message'].apply(nltk.word_tokenize)

In [274]:
tokenize(train_data)
train_data['Tokens'].head()

0    [@, MeNyrbie, @, Phil_Gahan, @, Chrisitv, http...
1    [advice, Talk, to, your, neighbours, family, t...
2    [Coronavirus, Australia, :, Woolworths, to, gi...
3    [My, food, stock, is, not, the, only, one, whi...
4    [Me, ,, ready, to, go, at, supermarket, during...
Name: Tokens, dtype: object

## c)

In [275]:
def remove_url(data):
    tokens_no_url = []

    # for the tokens of each row, remove all occurences of url
    for row in data['Tokens']:
        tokens_no_url.append([re.sub('^http', '', t) for t in row])

    # Replace our tokens with the url-removed version
    data['Tokens'] = tokens_no_url

In [276]:
remove_url(train_data)
train_data['Tokens'].head()

0    [@, MeNyrbie, @, Phil_Gahan, @, Chrisitv, s, :...
1    [advice, Talk, to, your, neighbours, family, t...
2    [Coronavirus, Australia, :, Woolworths, to, gi...
3    [My, food, stock, is, not, the, only, one, whi...
4    [Me, ,, ready, to, go, at, supermarket, during...
Name: Tokens, dtype: object

## d)

In [277]:
def remove_punctuation(data):
    tokens_no_punct = []
    # For the tokens of each row, remove all occurrences of punctuations (i.e. non-alphanumeric and non-whitespace)
    for row in data['Tokens']:
        tokens_no_punct.append([re.sub('[^\w\s]', '', t) for t in row])
    # Replace our tokens with the punctuation-removed version
    data['Tokens'] = tokens_no_punct

def convert_to_lowercase(data):
    lowercase_tokens = []
    # For the tokens of each row, convert all strings to lowercase
    for row in data['Tokens']:
        lowercase_tokens.append([t.lower() for t in row])
    # Replace our tokens with the lowercase version
    data['Tokens'] = lowercase_tokens

In [278]:
remove_punctuation(train_data)
convert_to_lowercase(train_data)
train_data['Tokens'].head()

0    [, menyrbie, , phil_gahan, , chrisitv, s, , tc...
1    [advice, talk, to, your, neighbours, family, t...
2    [coronavirus, australia, , woolworths, to, giv...
3    [my, food, stock, is, not, the, only, one, whi...
4    [me, , ready, to, go, at, supermarket, during,...
Name: Tokens, dtype: object

## e)

In [279]:
def stemming_tokens(data):
    ### Stemming our dataset using PorterStemmer
    stemmer = PorterStemmer()

    stemmed_tokens = []
    for row in data['Tokens']:
      stemmed_tokens.append([stemmer.stem(t) for t in row])

    data['Tokens'] = stemmed_tokens

In [280]:
stemming_tokens(train_data)
train_data['Tokens'].head()

0    [, menyrbi, , phil_gahan, , chrisitv, s, , tco...
1    [advic, talk, to, your, neighbour, famili, to,...
2    [coronaviru, australia, , woolworth, to, give,...
3    [my, food, stock, is, not, the, onli, one, whi...
4    [me, , readi, to, go, at, supermarket, dure, t...
Name: Tokens, dtype: object

## f)

In [281]:
def remove_stopwords(data):
    sw = stopwords.words('english')[:100]
    tokens_no_sw = []
    # For the tokens of each row, remove all occurrences of stopwords
    for row in data['Tokens']:
        tokens_no_sw.append([w for w in row if w not in sw])
    # Replace our tokens with the stopwords-removed version
    data['Tokens'] = tokens_no_sw

In [282]:
remove_stopwords(train_data)
train_data['Tokens'].head()

0    [, menyrbi, , phil_gahan, , chrisitv, s, , tco...
1    [advic, talk, neighbour, famili, exchang, phon...
2    [coronaviru, australia, , woolworth, give, eld...
3    [food, stock, not, onli, one, empti, , pleas, ...
4    [, readi, go, supermarket, dure, , covid19, ou...
Name: Tokens, dtype: object

## g)

In [296]:
def split_tokens_and_label(data):
    return data['Tokens'].to_numpy(), data['Sentiment'].to_numpy()

def override_fcn(doc):
  # We expect a list of tokens as input
  return doc

def convert_to_vector(X):
    # Count Vectorizer
    count_vec = CountVectorizer(
        analyzer='word',
        tokenizer= override_fcn,
        preprocessor= override_fcn,
        token_pattern= None)

    # Remember this output is a Scipy Sparse Array
    counts = count_vec.fit_transform(X)
    # print(counts.toarray())

    # # Print this mapping as dictionary
    # print(count_vec.vocabulary_)

    # Print the length of the vocabulary
    # print(f'The length of the vocabulary is {len(count_vec.vocabulary_)}')
    return counts

In [284]:
X_train, y_train = split_tokens_and_label(train_data)
X_train = convert_to_vector(X_train)

[[6 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [3 0 0 ... 0 0 0]
 ...
 [7 0 0 ... 0 0 0]
 [4 0 0 ... 0 0 0]
 [5 0 0 ... 0 0 0]]
The length of the vocabulary is 74225


## h) `train_data`


In [285]:
def fit_naive_bayes_model(X_data, y_data):
    nb = MultinomialNB()
    # Fit model to the data
    nb.fit(X_data, y_data)
    return nb

def predict(X_data, y_data, mode):
    nb = fit_naive_bayes_model(X_data, y_data)
    y_preds = nb.predict(X_data)
    print(f'Test accuracy with simple Naive Bayes on {mode} data: ', accuracy_score(y_data, y_preds))

In [286]:
predict(X_train, y_train, 'training')

Test accuracy with simple Naive Bayes on training data:  0.8212281000170097


## h) `test_data`

In [297]:
# Read the test data from CSV file
test_data = pd.read_csv('covid-tweets-test.csv')
test_data.columns = ['Label', 'Message', 'Sentiment']
test_data.head()
# Test our model by doing the same analysis to test_data
filter_missing_values(test_data)
tokenize(test_data)
remove_url(test_data)
remove_punctuation(test_data)
convert_to_lowercase(test_data)
stemming_tokens(test_data)
remove_stopwords(test_data)
X_test, y_test = split_tokens_and_label(test_data)
X_test = convert_to_vector(X_test)
predict(X_test, y_test, 'test')

Test accuracy with simple Naive Bayes on test data:  0.8667719852553976


# Part II

In [288]:
# bearer_token="AAAAAAAAAAAAAAAAAAAAABjFsgEAAAAAlpTaLAR1wJ2sut2HNk8oY2r9u28%3DRWpfy37hyg15PEYEQJAQwhjS9S3RasSkc1WV35KH1FhBPfAgjF"

In [289]:
# import tweepy as tw

# client = tw.Client(bearer_token=bearer_token)

In [290]:
# # ONLY RUN THIS CELL ONCE IN LAB SO AVOID GOING OVER THE RATE LIMIT FOR THIS ACCOUNT!

# #Collect tweets (here, I get only 20)
# search_words = '#toronto'

# response = client.search_recent_tweets(search_words, max_results=100)
# tweets = response.data

In [291]:
# print(f'Number of tweets: {len(tweets)}')

# # The result is an iterable
# for tweet in tweets:
#   print(tweet.text)

# # Could also use a list comprehension

In [292]:
# tweet_list = [tweet.text for tweet in tweets]
# tweet_text = pd.DataFrame(tweet_list, columns = ['tweet'])

# print(tweet_text.head(5))

In [293]:
# # Extract handles
# handle_regex = '@[A-Za-z|0-9]+'

# tweet_text['handles'] = tweet_text['tweet'].str.findall(handle_regex)
# print(tweet_text.head(10))

In [294]:
# # Define the file path to save the DataFrame
# file_path = 'twitter_data.txt'

# # Write the DataFrame to a text file
# tweet_text.to_csv(file_path, sep='\t', index=False)

# print(f"DataFrame has been successfully written to '{file_path}'.")